In [19]:
import numpy as np
import numpy.linalg as lin
from scipy.special import erf

## Implementation of Hartree-Fock 

The following code works only with 1s-type primitive gaussian functions

In [20]:
sto3g = {
    'H': {'exp': [0.168714, 0.623913, 3.42525], 'coeff': [0.444635, 0.535328, 0.154329]},
    'He': {'exp': [0.1612778, 0.6076837, 3.42525], 'coeff': [0.444635, 0.535328, 0.154329]},
    'C': {'exp': [0.109818, 0.405771, 2.22766], 'coeff': [0.444635, 0.535328, 0.154329]},
    'O': {'exp': [0.105469, 0.397389, 2.35044], 'coeff': [0.444635, 0.535328, 0.154329]}
}

In [21]:
elements = {"H": 1, "He": 2, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, "F": 9, "Ne": 10, 
            "Na": 11, "Mg": 12, "Al": 13, "Si": 14, "P": 15, "S": 16, "Cl": 17, "Ar": 18, "K": 19, 
            "Ca": 20, "Sc": 21, "Ti": 22, "V": 23, "Cr": 24, "Mn": 25, "Fe": 26, "Co": 27, "Ni": 28, 
            "Cu": 29, "Zn": 30, "Ga": 31, "Ge": 32, "As": 33, "Se": 34, "Br": 35, "Kr": 36, "Rb": 37, 
            "Sr": 38, "Y": 39, "Zr": 40, "Nb": 41, "Mo": 42, "Tc": 43, "Ru": 44, "Rh": 45, "Pd": 46, 
            "Ag": 47, "Cd": 48, "In": 49, "Sn": 50, "Sb": 51, "Te": 52, "I": 53, "Xe": 54, "Cs": 55, 
            "Ba": 56, "La": 57, "Ce": 58, "Pr": 59, "Nd": 60, "Pm": 61, "Sm": 62, "Eu": 63, "Gd": 64, 
            "Tb": 65, "Dy": 66, "Ho": 67, "Er": 68, "Tm": 69, "Yb": 70, "Lu": 71, "Hf": 72, "Ta": 73, 
            "W": 74, "Re": 75, "Os": 76, "Ir": 77, "Pt": 78, "Au": 79, "Hg": 80, "Tl": 81, "Pb": 82, 
            "Bi": 83, "Th": 90, "Pa": 91, "U": 92, "Np": 93, "Pu": 94, "Am": 95, "Cm": 96, "Bk": 97, 
            "Cf": 98, "Es": 99, "Fm": 100, "Md": 101, "No": 102, "Lr": 103}

In [22]:
# construct basis and cores

def read_xyz(filename, elements=elements, sto3g=sto3g):
    #if filename.endswith(".xyz"):
    with open(filename, 'r') as f:
        natoms = int(f.readline())

        # Skip comment line
        f.readline()

        cores = []
        atom_info = []

        # Read in data for each atom
        for i in range(natoms):
            line = f.readline().split()
            cores.append([elements[line[0]], np.array([float(line[1]), float(line[2]), float(line[3])]), line[0]])
            atom_info.append([np.array([float(line[1]), float(line[2]), float(line[3])]), line[0]])

    return cores, atom_info

def make_basis(atom_info):
    basis = []
    for data in atom_info:
        basis.append([data[0], sto3g[data[1]]["exp"], sto3g[data[1]]["coeff"]])
    return basis

#### Product of two gaussians

In [23]:
def gauss_product(R_A, R_B, a, b):
    K = np.exp(-a*b*lin.norm(R_A-R_B)**2/(a+b))
    R_p = (a*R_A + b*R_B)/(a+b)
    p = a + b
    return K, R_p, p          

In [24]:
def Fo(t):
    # see Szabo Ostlund p. 415
    # setup slightly adapted error function
    if t == 0:
        return 1
    else:
        return 0.5*np.sqrt((np.pi/t))*erf(np.sqrt(t))

In [25]:
def prefactor(a,b):
    return (2*a/np.pi)**(3/4) * (2*b/np.pi)**(3/4)

## One electron intergrals

In [26]:
def overlap(R_A, R_B, a, b):
    return prefactor(a,b) * (np.pi/(a+b))**(3/2) * np.exp(-a*b/(a+b)*lin.norm(R_A-R_B)**2)

In [27]:
def kinetic(R_A, R_B, a, b):
    diff = lin.norm(R_A-R_B)**2
    coeff = a*b/(a+b)
    return prefactor(a,b) * coeff*(3-2*coeff*diff)*(np.pi/(a+b))**(3/2)*np.exp(-coeff*diff)

In [28]:
def potential(charge, pos, R_A, R_B, a, b):
    K, R_p, p = gauss_product(R_A, R_B, a, b)
    return prefactor(a,b)*(-2)*np.pi/p*charge*K*Fo(p*lin.norm(R_p - pos)**2)

## Two electron integrals

In [29]:
def two_electron(R_A, R_B, R_C, R_D, a, b, c, d):
    K, R_p, p = gauss_product(R_A, R_B, a, b)
    G, R_Q, q = gauss_product(R_C, R_C, c, d)
    return prefactor(a,b)*prefactor(c,d)*2*np.pi**(5/2)/(p*q*np.sqrt(p+q)) * K * G * Fo(p*q/(p+q)*lin.norm(R_p-R_Q)**2)

## Core repulsion

In [30]:
def V_NN(cores):
    VN = 0
    for i, Z_1 in enumerate(cores):
        for j, Z_2 in enumerate(cores):
            if i == j:
                continue
            VN += Z_1[0] * Z_2[0] / lin.norm(Z_1[1] - Z_2[1])
    return 0.5 * VN

basis = np.array[[[R1], [exp1], [coef1]], [[R3], [exp2], [coef2]], [[R3], [exp3], [coef3]]]

cores = np.array([[Z1, [R1]], [Z2, [R2]], [Z3, [R3]]]

In [31]:
def single_electron_matrix(basis, num_gauss, cores):
    
    num_basis = len(basis)
    
    S = np.zeros((num_basis, num_basis)) # overlap matrix
    T = np.zeros((num_basis, num_basis)) # kinetic energy matrix
    V = np.zeros((num_basis, num_basis)) # potential energy matrix
    
    # Spater ausnutzen, dass Matrix hermitesch ist
    
    # Iterate through contracted functions
    for i in range(num_basis):
        basis_1 = basis[i]
        R_A = basis_1[0]
    
        for j in range(num_basis):
            basis_2 = basis[j]
            R_B = basis_2[0]
            
            E_kin = 0
            over = 0
            V_pot = 0
            
            # Compute Integrals with gaussians
            for k in range(num_gauss):
                a = basis_1[1][k]
                c_a = basis_1[2][k]
                
                for l in range(num_gauss):         
                    b = basis_2[1][l]
                    c_b = basis_2[2][l]
                    
                    E_kin += c_a * c_b * kinetic(R_A, R_B, a, b)
                    over += c_a * c_b * overlap(R_A, R_B, a, b)
                    
                    for kern in range(len(cores)):
                        Z = cores[kern][0]
                        pos = cores[kern][1]
                        
                        V_pot += c_a * c_b * potential(Z, pos, R_A, R_B, a, b)
                    
            T[i,j] = E_kin 
            S[i,j] = over
            V[i,j] = V_pot
    
    return S, T, V 

In [32]:
def two_electron_matrix(basis, num_gauss):
    num_basis = len(basis)
    
    tensor = np.zeros((num_basis, num_basis, num_basis, num_basis)) # two electron tensor
    
    # wiecores[0][0] kann ich Hermitizitat fur diesen Tensor ausnutzen
    
    # Iterate through contracted functions
    # I know this is not fast, as loops are very slow in python, vectrorize for bigger molecules!!!
    for i in range(num_basis):
        basis_1 = basis[i]
        R_A = basis_1[0]
        
        for j in range(num_basis):
            basis_2 = basis[j]
            R_B = basis_2[0]
            
            for k in range(num_basis):
                basis_3 = basis[k]
                R_C = basis_3[0]
                
                for l in range(num_basis):
                    basis_4 = basis[j]
                    R_D = basis_4[0]
                    
                    value = 0
                    for u in range(num_gauss):
                        a = basis_1[1][u]
                        c_a = basis_1[2][u]
                        
                        for v in range(num_gauss):
                            b = basis_2[1][v]
                            c_b = basis_2[2][v]
                            
                            for w in range(num_gauss):
                                c = basis_3[1][w]
                                c_c = basis_3[2][w]
                                
                                for x in range(num_gauss):
                                    d = basis_4[1][x]
                                    c_d = basis_4[2][x]
                                    
                                    value += c_a*c_b*c_c*c_d*two_electron(R_A, R_B, R_C, R_D, a, b, c, d)
                                    
                    tensor[i,j,k,l] = value
                                    
    return tensor

## Constuct Fock matrix

In [33]:
# Reads in Geometry and computes all relevant properties

cores, atom_info = read_xyz("H2.xyz")
basis = make_basis(atom_info)
num_gauss = 3

num_electrons = 2
num_occ_orbitals = num_electrons / 2 # weil RHF
num_basis = len(basis) # number of basis functions

# Makes initial guess for density matrix D
occupation = num_occ_orbitals/ num_basis # distribute the number of all RHF electrons (num_electrons / 2)
D = np.diag(np.full((1,num_basis), occupation)[0]) # same as np.diag([occupation for i in range(12)])

# Computes single electron matrices
S, T, V = single_electron_matrix(basis, num_gauss, cores)
H0 = T + V

# Computes Fock Matrix
tensor = two_electron_matrix(basis, num_gauss)

V_NN = V_NN(cores)

In [34]:
# The actual SCF procedure

def SCF_procedure(delta_E, S=S, H0=H0, tensor=tensor, D=D, V_NN=V_NN, num_basis=num_basis):
    
    HF_wavefunction = np.array([]) # Denstiy matrix * basis_functions
    
    # Compute Fock Matrix with density matrix
    
    # Constructs initial Fock matrix
    F = np.zeros((num_basis,num_basis))
    
    for i in range(num_basis):
        for j in range(num_basis):
            value = 0
            for k in range(num_basis):
                for l in range(num_basis):
                    value += D[k,l] * (2 *tensor[i,j,k,l] * tensor[i,k,j,l])                
            F[i,j] = H0[i,j] + value
            
    # Compute eigenvalues of Fock matrix
    # Output Energy from occ 
    # construct new density matrix if not converged and repeat process
    
    energy = 10 
    last_energy = 0
    iteration = 1
    max_iterations = 100
    
    while abs(energy - last_energy) > delta_E and iteration < max_iterations:
        
        print(f"\n Iteration {iteration} \n")
        print("\n Density matrix", D)
        print("\n Fock matrix", F)
        print(f"\n Energy in Ha: {energy} \t for Iteration {iteration} \n")
        
    energy = energy + V_NN # Final energy that includes core repulsion
    
    return energy, HF_wavefunction, iteration

## Diagonalize Fock-Matrix

In [35]:
evalS, U = lin.eig(S)
diagS = np.dot(U.T, np.dot(S,U))
diagS_minushalf = np.diag(np.diagonal(diagS)**(-0.5))
X = np.dot(U, np.dot(diagS_minushalf, U.T))